# tensorflow 주요 아키텍처

+ 고수준 API : 케라스, 에스티메이터API
+ 저수준 API : Python wrapper (tf.nn)
+ C++ Tensorflow : c++ 계층


### 데이터 준비(MNIST)

In [1]:
import tensorflow as tf

num_classes =10 
img_rows, img_cols = 28,28
num_channels =1
input_shape = (img_rows, img_cols, num_channels)

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/ 255.0, x_test/255.0

11493376/11490434 [==============================] - 0s 0us/step


tf.keras.datasets 모듈은 고전적인 데이터셋을 내려 받고 인스턴스화하기 위해 빠르게 접근할 수 있게 지원한다.

### 모델 구성

In [2]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()) # (28x28) --> (1 x 784)
model.add(tf.keras.layers.Dense(128, activation = 'relu')) # (1 x 784) x (784 * 128) --> (1* 128)
model.add(tf.keras.layers.Dense(num_classes, activation = 'softmax')) #(1 * 128) x (128 * num_classes) --> (1 * numclasses)

### 모델 훈련

In [3]:
model.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, epochs = 5, verbose = 1, validation_data = (x_test, y_test)) 

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6569 - accuracy: 0.8341 - val_loss: 0.3575 - val_accuracy: 0.9047
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3353 - accuracy: 0.9070 - val_loss: 0.2929 - val_accuracy: 0.9173
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2856 - accuracy: 0.9204 - val_loss: 0.2590 - val_accuracy: 0.9270
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2552 - accuracy: 0.9284 - val_loss: 0.2376 - val_accuracy: 0.9346
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2328 - accuracy: 0.9354 - val_loss: 0.2182 - val_accuracy: 0.9378


### 모델 구성 보기

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


# Tensorflow 2와 케라스

+ tensor: N차원 배열로 생각할 수 있음
+ 고정값: tf.constant
+ 변경되는 값: tf.Variable
----------------------------------------------------------------
- 타입: string, float32, float16, int8
- 형상: ()이면 스칼라고, (n)이면 크기가 n인 벡터, (n,m)이면 크기가 nxm인 2차원 행렬
- 순위: 차원 개수, 0이면 스칼라, 1이면 벡터, 2이면 2차원 행렬이다. 

### 텐서플로 그래프
입력과 출력으로 텐서를 사용, 그래프를 사용해 연산을 표현한다.
+ 장점
  > CPU에서 일부 연산을 실행하고 GPU에서 남은 연산을 실행한다.

  > 분산 모델의 경우 그래프의 다양한 부분을 여러 다른 컴퓨터에서 실행한다.
  
  > 불필요한 연산을 피하기 위해 그래프를 최적화해 계산 성능을 개선한다.


In [5]:
a  = tf.constant([1,2,3])
b = tf.constant([0,0,1])
c = tf.add(a,b)
print(c)

tf.Tensor([1 2 4], shape=(3,), dtype=int32)


### 텐서플로2 그래프 생성

In [6]:
def compute(a,b,c):
  d = a*b+c
  e = a*b*c
  return d,e

tensorflow2에서는 모든 연산이 정의되는 시점에 실행됨

### 텐서플로 오토프래프와 tf.function

In [7]:
@tf.function
def compute(a,b,c):
  d = a*b+c
  e = a*b*c
  return d,e

@tf.function을 함으로써 tensorflow2에서 그래프를 투명하게 생성한다. 
--> 그래프를 최적화한다.

#### 오토그래프를 사용할때
+ 모델을 다른 기기로 내보내야 할 때
+ 성능이 무엇보다 중요하고 그래프 최적화를 통해 속도 개선이 가능할 때

### 자동 미분
+ 그래프의 장점으로 자동 미분이 가능하다. --> "Gradient tape"


### 그래디언트 테이프를 사용해 오차 역전파
+ A*X = B의 공식을 풀어야할 때--> abs(A*X -B)의 값을 0에 가깝게 해야함

In [9]:
A, B = tf.constant(3.0), tf.constant(6.0)
X = tf.Variable(20.0)
loss = tf.math.abs(A*X -B)

In [13]:
def train_step():
  with tf.GradientTape() as tape:
    loss = tf.math.abs(A * X -B) #0과 차이나는 정도
  dx = tape.gradient(loss, X) #X의 값과 loss에 대해 미분
  print('X = {:.2f}, dx = {:.2f}'.format(X.numpy(), dx))
  X.assign(X-dx)

for i in range(7):
  train_step()

X = 20.00, dx = 3.00
X = 17.00, dx = 3.00
X = 14.00, dx = 3.00
X = 11.00, dx = 3.00
X = 8.00, dx = 3.00
X = 5.00, dx = 3.00
X = 2.00, dx = 0.00


### tf.function 작동방식

+ 작업에 맞는 그래프를 생성한다. 한번 생성하면 캐시에 저장하기 때문에 같은 함수가 호출 될 때 다시 그래프를 생성하지 않는다.



In [14]:
@tf.function
def identity(x):
  print('Creating graph!')
  return x

x1 = tf.random.uniform((10,10))
x2 = tf.random.uniform((10,10)) #담고 있는 변수는 다르지만 같은 기능을 하는 그래프임

identity(x1)
identity(x2)



Creating graph!


<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0.549209  , 0.35636032, 0.27082717, 0.4492725 , 0.13679028,
        0.18062401, 0.43467033, 0.6780201 , 0.7991804 , 0.7365551 ],
       [0.8615266 , 0.59680843, 0.6459272 , 0.8726394 , 0.3307129 ,
        0.30378616, 0.08883762, 0.4081745 , 0.7197554 , 0.10173333],
       [0.68461096, 0.926479  , 0.6140243 , 0.02766776, 0.6120273 ,
        0.7479558 , 0.6360487 , 0.22396088, 0.05399036, 0.8753573 ],
       [0.24539876, 0.4239509 , 0.20855927, 0.31921172, 0.83510244,
        0.5447372 , 0.5877503 , 0.47429264, 0.8954402 , 0.00107813],
       [0.04495502, 0.9841813 , 0.6677946 , 0.4474957 , 0.9946673 ,
        0.05413079, 0.09171927, 0.40901828, 0.67850375, 0.32516074],
       [0.90853477, 0.23434234, 0.9049883 , 0.5356443 , 0.9953798 ,
        0.5434228 , 0.16219926, 0.5707768 , 0.78103995, 0.74849296],
       [0.76487386, 0.23701608, 0.00766098, 0.19620574, 0.5720842 ,
        0.41596258, 0.9258369 , 0.77993023, 0.7341188 , 0.37

x2의 'Creating graph'는 출력 되지 않음

In [15]:
x3 = tf.random.uniform((10,10), dtype = tf.float16) #그래프 기능에 조금만 달라져도 생성이 실행된다.
identity(x3)

Creating graph!


<tf.Tensor: shape=(10, 10), dtype=float16, numpy=
array([[0.5713 , 0.6533 , 0.836  , 0.7656 , 0.5996 , 0.5625 , 0.4268 ,
        0.2715 , 0.9434 , 0.9854 ],
       [0.1387 , 0.1953 , 0.6367 , 0.03906, 0.0908 , 0.9893 , 0.3525 ,
        0.4902 , 0.675  , 0.2412 ],
       [0.3535 , 0.742  , 0.     , 0.2656 , 0.703  , 0.2354 , 0.626  ,
        0.674  , 0.0928 , 0.495  ],
       [0.54   , 0.3672 , 0.8965 , 0.2334 , 0.9854 , 0.04883, 0.3604 ,
        0.5    , 0.1855 , 0.415  ],
       [0.3262 , 0.7637 , 0.54   , 0.5605 , 0.4268 , 0.539  , 0.1172 ,
        0.0752 , 0.124  , 0.8145 ],
       [0.3994 , 0.42   , 0.7744 , 0.749  , 0.4072 , 0.8213 , 0.552  ,
        0.1572 , 0.802  , 0.661  ],
       [0.9473 , 0.3857 , 0.4453 , 0.4277 , 0.4912 , 0.4229 , 0.88   ,
        0.3223 , 0.754  , 0.577  ],
       [0.992  , 0.948  , 0.1777 , 0.296  , 0.1436 , 0.9375 , 0.3604 ,
        0.583  , 0.99   , 0.03125],
       [0.3691 , 0.169  , 0.4258 , 0.3018 , 0.8516 , 0.0674 , 0.06836,
        0.6836 , 0.837 

### 텐서플로 2에서의 변수

In [17]:
a = tf.Variable(3, name = 'my_var')
print(a)

<tf.Variable 'my_var:0' shape=() dtype=int32, numpy=3>


In [18]:
a.assign(a+1)
print(a.numpy())

4


In [19]:
b = a+1
print(b)

tf.Tensor(5, shape=(), dtype=int32)
